In [1]:
#import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt

In [2]:
# ****************************************************************************************************************

In [3]:
def structureDataFile(df):
    cutoff=0
    df_copy = df.copy()
    filename='RankFinal_cutoff'+str(cutoff)+'.csv'
    
    df_copy['Row']=np.ceil(df_copy['barcodeNum_REDI']/48).astype(int)
    df_copy['Column']= df_copy['barcodeNum_REDI'] - (df_copy['Row']-1)*48
    
    return df_copy

    

In [4]:
# function to evaluate input df(rowname & colname) and get the position in ColonyPlateDf and return the input dataframe (df) 
# by adding the column 'Matched' (0 - not available, 1- available in original plate)

def evaluateColony(df,rowname,colname,ColonyPlateDF):
    b=np.zeros(shape=(len(df),1))
    for i in range(len(df)):
            cond1 = ColonyPlateDF['Row'] == df[rowname][i]
            cond2 = ColonyPlateDF['Column']== df[colname][i]
            b[[i]] = (ColonyPlateDF[cond1 & cond2]['Positions']).as_matrix()
    
    B = pd.DataFrame(b)
    
    FinalDF = pd.concat([df,B],axis=1,sort=False)
    
    FinalDF.rename(columns={0:'Positions'},inplace=True)
    FinalDF['Positions'] = FinalDF['Positions'].astype(np.int64)
    
    #Total = FinalDF.shape[0]
    #FinalDF['MatchCount'] = 
    
    return(FinalDF)      
    
    

In [5]:
# Function to get the True / False positives and accuracy 
def metricsSummary(df):
    df1 = df[['Plate','AS_FWD','AS_REV','AS_REDI','Positions']].groupby(['Plate','AS_FWD','AS_REV','AS_REDI'],as_index= False).agg({'Positions': "count"}).sort_values(by=(['AS_FWD','AS_REV','AS_REDI']),ascending=False)
    df1.rename(columns={'Positions': 'Total'},inplace=True)
    
    df2 = df[['Plate','AS_FWD','AS_REV','AS_REDI','Positions']].groupby(['Plate','AS_FWD','AS_REV','AS_REDI'],as_index= False).agg({'Positions': sum})
    df2.rename(columns={'Positions': 'TruePositive'},inplace=True)
    
    df3 = pd.merge(df1,df2,on=['Plate','AS_FWD','AS_REV','AS_REDI'],how='inner')
    df3['FalsePositive']=df3['Total'] - df3['TruePositive']
    
    df3['Accuracy'] = round(df3['TruePositive'] / df3['Total'],2)
    
    df3['AS_Combinations']=df3['AS_FWD'].astype(str) + ':' + df3['AS_REV'].astype(str) + ':' + df3['AS_REDI'].astype(str)
    
    return df3
    

In [6]:
def PlotPlateMatrics(df,plate):
    df_copy = df[df['Plate']==plate].copy()
    dfPlot = df[['AS_Combinations','TruePositive','FalsePositive']]
    dfPlot.set_index('AS_Combinations',inplace= True) 
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    
    fig= plt.figure(figsize=(20,10))

    ax = fig.add_subplot(111) 
    dfPlot.plot.bar(rot=0,ax=ax)
    
    plt.xticks(rotation='vertical')
    
    ax.set(Title='Alignment Score: '+ plate ,xlabel="FWD: REV: REDI", ylabel="Barcode counts")
    plt.tight_layout()
    
    
    fig.savefig('Alignment Score Plate4', dpi=fig.dpi)
    plt.show()
    

In [7]:
def bestMatchedAS(df,Top_AS_FWD,Top_AS_REV,Top_AS_REDI):
    cond1 = df['AS_FWD'].isin(Top_AS_FWD)
    cond2 = df['AS_REV'].isin(Top_AS_REV)
    cond3 = df['AS_REDI'].isin(Top_AS_REDI)
    DF_TopAS = df[cond1 & cond2 & cond3].reset_index(drop=True)
    return DF_TopAS


In [8]:
def SplitDataFrame(df,PlateNo):
    df_copy = df.copy()
    #SplitDF = df_copy[df_copy['Plate'] == PlateList[PlateNo-1]].reset_index(drop=True)
    SplitDF = df_copy[df_copy['Plate'] == PlateList[PlateNo]].reset_index(drop=True)
    dataset = SplitDF[['Plate','headers_REDI','read_id']].groupby(['Plate','headers_REDI'],as_index=False).count().sort_values(by='headers_REDI',ascending=True)
    #y['headers_REDI']=y['headers_REDI']+1
    return dataset

In [9]:
def GetPlateDataFrame(df):
    df_copy = df.copy()
    DF_PLATE = pd.DataFrame()
    PlateName = df_copy['Plate'][0]
    read_id = 0
    n = df_copy.shape[0]
    
    listBarcodes = df['headers_REDI'].tolist()
    missingBarcode = []
    for i in range(0,1536):
    #for i in range(0,max(listBarcodes)):
    #for i in listBarcodes:
        if (i not in listBarcodes):
            missingBarcode.append(i)
    
    for i in missingBarcode:
        DF = pd.DataFrame({'Plate':PlateName,
                    'headers_REDI':[i],
                    'read_id':read_id},index = [n])
        DF_PLATE = pd.concat([DF_PLATE,DF])
        n = n+1
   
    DF_PLATE = pd.concat([df_copy,DF_PLATE]).sort_values('headers_REDI').reset_index(drop=True)
    return(DF_PLATE)

In [10]:
def GetPlotDF(df):
    df_copy = df.copy()
    b=np.zeros(shape=(32,48))
    #b=np.zeros(shape=(29,48))
    min = 0
    
    for i in range(0,32):
    #for i in range(0,28):
        max = min+48 # 48
        #print(min,max)
        cond1 = df_copy['headers_REDI']>= min
        cond2 = df_copy['headers_REDI']< max
        b[[i]] = (df_copy[cond1 & cond2]['read_id']).as_matrix()
        min=max
    
    B = pd.DataFrame(b)
    return(B)


In [11]:
# Returns matrix of selected genes
def highlightGene(df,plate):
    df_copy = df.copy()
    df_copy = df_copy[df_copy['Plate']==plate]
    df_copy = (df_copy[['Column','Row']].as_matrix())- 0.5
    return df_copy

In [12]:
def PlotHeatMap_old(df,PlateName, GenePlateDF,DFBarcodePlate):
    dfPlot = df
    import matplotlib.pyplot as plt
    from matplotlib import cm
    import numpy as np
    import matplotlib.colors as mcolors

    fig= plt.figure(figsize=(20,10))

    ax = fig.add_subplot(111) 
    #ax.set_title(PlateName,loc='left')
    
    #xticks = Plate1Plot.columns.tolist()
    xticks = dfPlot.columns.tolist()
    xticks.append(len(xticks)) # to add 48th column name
       
    #ax.set_xticks(Plate1Plot.columns.tolist())
    ax.set_xticks(xticks)

    #yticks = Plate1Plot.index.tolist()
    yticks = dfPlot.index.tolist()
    yticks.append(len(yticks)) # to add 32nd index name
       
    #ax.set_xticks(Plate1Plot.columns.tolist())
    ax.set_yticks(yticks)
    
    #ax.set_yticks(Plate1Plot.index.tolist())
    ax.xaxis.tick_top()
    #cax = plt.imshow(dfPlot,cmap=cm.coolwarm, interpolation='nearest')
    cmap, norm = mcolors.from_levels_and_colors([0, 0.5, 1], ['black', 'white'])
    
    extent=(0.5,48.5,32.5,0.5)
    cax = plt.imshow(dfPlot,cmap=cmap, interpolation='nearest',extent=extent, norm=norm)
    #cax = plt.imshow(dfPlot,cmap=cmap, interpolation='nearest', norm=norm)
    
    #ax.set_xlim([1, 48])
    #ax.set_ylim([32,1])
    #ax.grid(color='r', linestyle='-', linewidth=2)

    # Add colorbar, make sure to specify tick locations to match desired ticklabels 
    #cbar = fig.colorbar(cax, ticks=[-1, 0, 28]) 
    #cbar.ax.set_yticklabels(['Low', 'Medium', 'High'])# vertically oriented colorbar 

    plt.minorticks_on()
    plt.grid(which='major', linestyle = '-', linewidth='0.5', color='white')
    plt.grid(which='minor', linestyle = '-', linewidth='0.25', color='white')
    
    
    plt.tick_params(which = 'both',
                   top = 'on',
                   left = 'on',
                   right = 'on',
                   bottom = 'on')
    
    # To highlight the Gene
    from matplotlib.patches import Rectangle
    
    print(PlateName)
    #GeneMatrix = highlightGene(RankFinal,'Alanine -10')
    
    #Colony = CPPlate4[CPPlate4['Positions'] == 0].reset_index()
    #CPPlate4 = CPPlate4[CPPlate4['Positions']=0]
    
    '''
        Plate_FP = DFBarcodePlate[DFBarcodePlate['Positions'] ==0]
        Plate_TP = DFBarcodePlate[DFBarcodePlate['Positions'] > 0]

        TP = highlightGene(Plate_TP,PlateName)
        FP = highlightGene(Plate_FP,PlateName)
   
        for i in range(0,TP.shape[0]):
            x=TP[i,0]
            y=TP[i,1]
            ax.add_patch(Rectangle((x,y), 1, 1, fill=False, edgecolor='green', lw=3)) # 28,2    
    
        for i in range(0,FP.shape[0]):
            x=FP[i,0]
            y=FP[i,1]
            ax.add_patch(Rectangle((x,y), 1, 1, fill=False, edgecolor='red', lw=3)) # 28,2  
    '''
      #GenePlateDF = GeneBCMatchedPlate4
    GenePlate = highlightGene(GenePlateDF,PlateName)#'Alanine -4') #
    
    for i in range(0,GenePlate.shape[0]):
        x=GenePlate[i,0] + .2
        y=GenePlate[i,1] + .2
        #ax.add_patch(Rectangle((x,y), .6, .6, fill=True, edgecolor='green', lw=3)) # 28,2          
        ax.add_patch(Rectangle((x,y), .6, .6, fill=True, facecolor='green', lw=3)) # 28,2 
    
    BestGeneMarker = highlightGene(RankFinal,PlateName)
    for i in range(0,BestGeneMarker.shape[0]):
            x=BestGeneMarker[i,0]
            y=BestGeneMarker[i,1]
            ax.add_patch(Rectangle((x,y), 1, 1, fill=False, edgecolor='red', lw=3)) # 28,2    
        
    plt.tight_layout()
    
    
    fig.savefig(PlateName, dpi=fig.dpi)
    plt.show()

In [13]:
def PlotHeatMap(df,PlateName, GenePlateDF,DFBarcodePlate,topGenes):
    dfPlot = df
    import matplotlib.pyplot as plt
    from matplotlib import cm
    import numpy as np
    import matplotlib.colors as mcolors

    fig= plt.figure(figsize=(20,10))

    ax = fig.add_subplot(111) 
    #ax.set_title(PlateName,loc='left')
    
    #xticks = Plate1Plot.columns.tolist()
    xticks = dfPlot.columns.tolist()
    xticks.append(len(xticks)) # to add 48th column name
       
    #ax.set_xticks(Plate1Plot.columns.tolist())
    ax.set_xticks(xticks)

    #yticks = Plate1Plot.index.tolist()
    yticks = dfPlot.index.tolist()
    yticks.append(len(yticks)) # to add 32nd index name
       
    #ax.set_xticks(Plate1Plot.columns.tolist())
    ax.set_yticks(yticks)
    
    #ax.set_yticks(Plate1Plot.index.tolist())
    ax.xaxis.tick_top()
    #cax = plt.imshow(dfPlot,cmap=cm.coolwarm, interpolation='nearest')
    cmap, norm = mcolors.from_levels_and_colors([0, 0.5, 1], ['black', 'white'])
    
    extent=(0.5,48.5,32.5,0.5)
    cax = plt.imshow(dfPlot,cmap=cmap, interpolation='nearest',extent=extent, norm=norm)
    #cax = plt.imshow(dfPlot,cmap=cmap, interpolation='nearest', norm=norm)
    
    #ax.set_xlim([1, 48])
    #ax.set_ylim([32,1])
    #ax.grid(color='r', linestyle='-', linewidth=2)

    # Add colorbar, make sure to specify tick locations to match desired ticklabels 
    #cbar = fig.colorbar(cax, ticks=[-1, 0, 28]) 
    #cbar.ax.set_yticklabels(['Low', 'Medium', 'High'])# vertically oriented colorbar 

    plt.minorticks_on()
    plt.grid(which='major', linestyle = '-', linewidth='0.5', color='white')
    plt.grid(which='minor', linestyle = '-', linewidth='0.25', color='white')
    
    
    plt.tick_params(which = 'both',
                   top = 'on',
                   left = 'on',
                   right = 'on',
                   bottom = 'on')
    
    # To highlight the Gene
    from matplotlib.patches import Rectangle
    
    print(PlateName)
    #GeneMatrix = highlightGene(RankFinal,'Alanine -10')
    
    #Colony = CPPlate4[CPPlate4['Positions'] == 0].reset_index()
    #CPPlate4 = CPPlate4[CPPlate4['Positions']=0]
    
    '''
        Plate_FP = DFBarcodePlate[DFBarcodePlate['Positions'] ==0]
        Plate_TP = DFBarcodePlate[DFBarcodePlate['Positions'] > 0]

        TP = highlightGene(Plate_TP,PlateName)
        FP = highlightGene(Plate_FP,PlateName)
   
        for i in range(0,TP.shape[0]):
            x=TP[i,0]
            y=TP[i,1]
            ax.add_patch(Rectangle((x,y), 1, 1, fill=False, edgecolor='green', lw=3)) # 28,2    
    
        for i in range(0,FP.shape[0]):
            x=FP[i,0]
            y=FP[i,1]
            ax.add_patch(Rectangle((x,y), 1, 1, fill=False, edgecolor='red', lw=3)) # 28,2  
    '''
      #GenePlateDF = GeneBCMatchedPlate4
    GenePlate = highlightGene(GenePlateDF,PlateName)#'Alanine -4') #
    
    for i in range(0,GenePlate.shape[0]):
        x=GenePlate[i,0] + .2
        y=GenePlate[i,1] + .2
        #ax.add_patch(Rectangle((x,y), .6, .6, fill=True, edgecolor='green', lw=3)) # 28,2          
        ax.add_patch(Rectangle((x,y), .6, .6, fill=True, facecolor='green', lw=3)) # 28,2 
    
    BestGeneMarker = highlightGene(topGenes,PlateName)
    for i in range(0,BestGeneMarker.shape[0]):
            x=BestGeneMarker[i,0]
            y=BestGeneMarker[i,1]
            ax.add_patch(Rectangle((x,y), 1, 1, fill=False, edgecolor='yellow', lw=3)) # 28,2    
        
    plt.tight_layout()
    
    
    fig.savefig(PlateName, dpi=fig.dpi)
    plt.show()

In [14]:
def TopRankGene(dfgene,cutoff):
    plateName = dfgene['Plate'][0]
    
    filename='RankGeneFinal_cutoff'+str(cutoff)+ plateName +'.csv'
    print(filename)

    dfBCsummary = dfgene[['Plate','barcodeNum_REDI','read_id']].groupby(['Plate','barcodeNum_REDI'],as_index=False).count().sort_values(by='read_id',ascending=False)
    dfBCsummary.rename(columns= {'read_id':'Total'}, inplace=True)
    #dfBCsummary.head()
    dfBCGenesummary = dfgene[['Plate','barcodeNum_REDI','GeneName','read_id']].groupby(['Plate','barcodeNum_REDI','GeneName'],as_index=False).count().sort_values(by='read_id',ascending=False)
    dfBCGenesummary.rename(columns= {'read_id':'GeneCount'}, inplace=True)
    #dfBCGenesummary.head()
    RankGeneDF = pd.merge(dfBCGenesummary,dfBCsummary, on=['Plate','barcodeNum_REDI'], how = 'inner')
    RankGeneDF['percentage']= ((RankGeneDF['GeneCount']/RankGeneDF['Total'])*100)
    
    RankGeneDFFinal= RankGeneDF[RankGeneDF['GeneCount']>cutoff].sort_values('percentage',ascending=False)
    l = list(range(1, len(RankGeneDFFinal)+1))
    RankGeneDFFinal['Rank'] = l
    RankGeneDFFinal['Row']=np.ceil(RankGeneDFFinal['barcodeNum_REDI']/48).astype(int)
    RankGeneDFFinal['Column']= RankGeneDFFinal['barcodeNum_REDI'] - (RankGeneDFFinal['Row']-1)*48
    
    RankGeneDFFinal = pd.merge(RankGeneDFFinal,GeneDF[['headers_GENES','GeneName']],on='GeneName', how='inner')
    #cols = RankGeneDFFinal.columns
    cols1 = ['Plate', 'barcodeNum_REDI', 'GeneName',  'headers_GENES', 'GeneCount', 'Total',
               'percentage', 'Rank', 'Row', 'Column']
    RankGeneDFFinal = RankGeneDFFinal[cols1]
    RankGeneDFFinal.to_csv(filename, index = False)
    return RankGeneDFFinal


In [15]:
def createHist(indices, start_range, end_range):
    nums = range(start_range,end_range)
    hist = [0]*end_range #highest quality you can have 
    for i in indices:
        for n in nums:
            if i==n:
                hist[n] +=1
    return hist